In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [5]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
sum_ = np.log(data_train.groupby(['item_id'])['user_id'].count()).sum()

In [8]:
weights = np.log(data_train.groupby(['item_id'])['user_id'].count())/sum_

Проверим правильно ли посчитали вероятности

In [9]:
weights.sum().round(5) == 1

True

In [12]:
items_weights = pd.DataFrame({'weight' : weights.array}, index=weights.index )

In [10]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights.index, size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [15]:
%%time

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

Wall time: 3.18 s


In [18]:
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1793907, 907033, 6773144, 995110, 13653496]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1071077, 9831555, 945774, 993247, 15686893]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [22]:
predict = pd.read_csv('predictions_basic.csv')
predict .head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [23]:
import re

for col in predict.columns[1:]:
    predict[col] = [np.array(re.findall('\d+', x), dtype=np.uintc) for x in predict[col]]

In [24]:
predict.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [26]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_array = np.array(bought_list)

    recommended_array = np.array(recommended_list[:k])

    bought_array = bought_array  # Тут нет [:k] !!

    flags = np.isin(bought_array, recommended_array)

    precision = flags.sum() / recommended_array.shape[0]

    return precision

In [27]:
fact = predict['actual']
for col in predict.columns[2:]:
    list_ = []
    for rec, act in zip(predict[col],fact):
        list_.append(precision_at_k(rec, act, k=5))
    prec=pd.Series(list_).mean()
    print(f'Для модели {col} средний precision_at_k равен {prec}')

Для модели random_recommendation средний precision_at_k равен 0.0005876591576885406
Для модели popular_recommendation средний precision_at_k равен 0.15523996082272082
Для модели itemitem средний precision_at_k равен 0.033594515181195064
Для модели cosine средний precision_at_k равен 0.03525954946131262
Для модели tfidf средний precision_at_k равен 0.03614103819784544
Для модели own_purchases средний precision_at_k равен 0.1799869409076044


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code